# TP 6 : Docker pour les Pipelines de Traitement de Données

## Objectifs

Ce TP présente la conteneurisation avec Docker pour construire des pipelines de traitement de données reproductibles et évolutifs. Vous apprendrez à empaqueter des applications, gérer des environnements multi-conteneurs et déployer des flux de traitement de données.

### Objectifs pédagogiques
* Comprendre l'architecture Docker et les concepts de conteneurisation
* Écrire des Dockerfiles pour empaqueter des applications Python
* Utiliser Docker Compose pour l'orchestration multi-conteneurs
* Implémenter des pipelines de données avec des volumes partagés
* Construire des modèles producteur-consommateur avec des files de messages
* Connecter des applications à des bases de données dans des conteneurs
* Implémenter des architectures frontend-backend
* Déployer et mettre à l'échelle des applications de traitement de données

### Prérequis
* Avoir terminé le TP 5 (Apache Spark)
* Docker Desktop installé ([Guide d'installation](https://docs.docker.com/get-docker/))
* Compréhension de base des commandes Linux
* Fondamentaux de la programmation Python

### Installation

Vérifiez que Docker est installé :
```bash
docker --version
docker-compose --version
```

### Aperçu des exercices

| Exercice | Sujet | Difficulté |
|----------|-------|------------|
| 1 | Fondamentaux de Docker et commandes de base | ★ |
| 2 | Écriture de Dockerfiles pour applications Python | ★ |
| 3 | Docker Compose pour applications multi-conteneurs | ★★ |
| 4 | Pipelines de données avec volumes partagés | ★★ |
| 5 | Producteur-Consommateur avec files de messages | ★★ |
| 6 | Intégration Application-Base de données | ★★ |
| 7 | Architectures Frontend-Backend | ★★★ |
| 8 | Mise à l'échelle et surveillance des conteneurs | ★★★ |

---

## Exercice 1 : Fondamentaux de Docker et commandes de base [★]

### Architecture Docker

Docker utilise une architecture client-serveur :

```
┌─────────────────────────────────────────────────────────────┐
│                     Hôte Docker                              │
│  ┌─────────────┐    ┌─────────────────────────────────────┐ │
│  │   Client    │    │          Démon Docker              │ │
│  │   Docker    │◄──►│  ┌─────────┐  ┌─────────┐           │ │
│  │   (CLI)     │    │  │Conteneur│  │Conteneur│           │ │
│  └─────────────┘    │  │    1    │  │    2    │           │ │
│                     │  └─────────┘  └─────────┘           │ │
│                     │       │            │                 │ │
│                     │  ┌────┴────────────┴────┐           │ │
│                     │  │      Images          │           │ │
│                     │  └─────────────────────┘           │ │
│                     └─────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────────┘
```

### Concepts clés

- **Image** : Modèle en lecture seule avec des instructions pour créer un conteneur
- **Conteneur** : Instance exécutable d'une image
- **Dockerfile** : Fichier texte avec des instructions pour construire une image
- **Registre** : Stockage pour les images Docker (ex. Docker Hub)

### Commandes Docker de base

Exécutez les commandes suivantes dans votre terminal pour vous familiariser avec Docker :

```bash
# Vérifier la version de Docker
docker --version

# Afficher les informations système
docker info

# Lister les images disponibles
docker images

# Lister les conteneurs en cours d'exécution
docker ps

# Lister tous les conteneurs (y compris arrêtés)
docker ps -a
```

### Exécuter votre premier conteneur

```bash
# Exécuter un simple conteneur hello-world
docker run hello-world

# Exécuter un conteneur Python interactif
docker run -it python:3.10 python

# Exécuter un conteneur avec une commande spécifique
docker run python:3.10 python -c "print('Bonjour depuis Docker !')"

# Exécuter un conteneur en arrière-plan (mode détaché)
docker run -d --name mon_python python:3.10 sleep 60

# Arrêter un conteneur en cours d'exécution
docker stop mon_python

# Supprimer un conteneur
docker rm mon_python
```

### Cycle de vie d'un conteneur

```
┌─────────┐   docker run   ┌─────────┐   docker stop   ┌─────────┐
│  Créé   │───────────────►│ En cours│────────────────►│ Arrêté  │
└─────────┘                └─────────┘                 └─────────┘
     │                          │                           │
     │                          │ docker pause              │
     │                          ▼                           │
     │                    ┌─────────┐                       │
     │                    │ En pause│                       │
     │                    └─────────┘                       │
     │                                                      │
     └──────────────────────────────────────────────────────┘
                        docker rm
```

```bash
# Afficher les logs d'un conteneur
docker logs <container_id>

# Exécuter une commande dans un conteneur en cours d'exécution
docker exec -it <container_id> bash

# Copier des fichiers vers/depuis un conteneur
docker cp fichier_local.txt <container_id>:/chemin/dans/conteneur/
docker cp <container_id>:/chemin/dans/conteneur/fichier.txt ./fichier_local.txt

# Afficher l'utilisation des ressources des conteneurs
docker stats
```

### Questions - Exercice 1

**Q1.1** Exécutez un conteneur Python qui affiche la version de Python du système, le nom du système d'exploitation et la date/heure actuelle. Capturez la sortie.

**Q1.2** Exécutez un conteneur Ubuntu de manière interactive. À l'intérieur du conteneur :
- Mettez à jour la liste des paquets
- Installez `curl`
- Téléchargez une page web
- Quittez le conteneur

**Q1.3** Exécutez trois conteneurs en mode détaché avec des noms différents. Utilisez `docker ps` pour vérifier qu'ils fonctionnent, puis arrêtez et supprimez-les tous en utilisant une seule commande chacun.

---

## Exercice 2 : Écriture de Dockerfiles pour applications Python [★]

### Bases du Dockerfile

Un Dockerfile est un script contenant des instructions pour construire une image Docker.

### Instructions Dockerfile courantes

| Instruction | Description |
|-------------|-------------|
| `FROM` | Image de base à partir de laquelle démarrer |
| `WORKDIR` | Définir le répertoire de travail |
| `COPY` | Copier des fichiers de l'hôte vers l'image |
| `RUN` | Exécuter des commandes pendant la construction |
| `ENV` | Définir des variables d'environnement |
| `EXPOSE` | Documenter les ports écoutés par le conteneur |
| `CMD` | Commande par défaut au démarrage du conteneur |
| `ENTRYPOINT` | Configurer le conteneur comme exécutable |

### Exemple : Application Python simple

Créez un fichier `app.py` :

```python
# app.py
import sys
import platform
from datetime import datetime

def main():
    print(f"Version Python : {sys.version}")
    print(f"Plateforme : {platform.platform()}")
    print(f"Heure actuelle : {datetime.now()}")
    print("Bonjour depuis Docker !")

if __name__ == "__main__":
    main()
```

Créez un `Dockerfile` :

```dockerfile
# Utiliser l'image Python officielle comme base
FROM python:3.10-slim

# Définir le répertoire de travail
WORKDIR /app

# Copier le code de l'application
COPY app.py .

# Définir la commande par défaut
CMD ["python", "app.py"]
```

Construire et exécuter :

```bash
# Construire l'image
docker build -t mon-app-python .

# Exécuter le conteneur
docker run mon-app-python
```

### Exemple : Application Python avec dépendances

Créez `requirements.txt` :

```
pandas==2.0.0
numpy==1.24.0
requests==2.28.0
```

Créez `data_processor.py` :

```python
import pandas as pd
import numpy as np

def process_data():
    # Créer des données exemple
    data = {
        'nom': ['Alice', 'Bob', 'Charlie', 'Diana'],
        'valeur': np.random.randint(1, 100, 4)
    }
    df = pd.DataFrame(data)
    
    print("Résultats du traitement des données :")
    print(df)
    print(f"\nSomme : {df['valeur'].sum()}")
    print(f"Moyenne : {df['valeur'].mean():.2f}")

if __name__ == "__main__":
    process_data()
```

`Dockerfile` optimisé :

```dockerfile
FROM python:3.10-slim

WORKDIR /app

# Copier d'abord les requirements (pour une meilleure mise en cache des couches)
COPY requirements.txt .

# Installer les dépendances
RUN pip install --no-cache-dir -r requirements.txt

# Copier le code de l'application
COPY data_processor.py .

CMD ["python", "data_processor.py"]
```

### Constructions multi-étapes

Les constructions multi-étapes aident à créer des images de production plus légères :

```dockerfile
# Étape de construction
FROM python:3.10 AS builder

WORKDIR /app

COPY requirements.txt .
RUN pip install --user --no-cache-dir -r requirements.txt

# Étape de production
FROM python:3.10-slim

WORKDIR /app

# Copier les paquets installés depuis le builder
COPY --from=builder /root/.local /root/.local

# S'assurer que les scripts dans .local sont utilisables
ENV PATH=/root/.local/bin:$PATH

COPY app.py .

CMD ["python", "app.py"]
```

### Bonnes pratiques pour les Dockerfiles

1. **Utiliser des tags d'image spécifiques** : `python:3.10-slim` au lieu de `python:latest`
2. **Ordonner les instructions par fréquence de changement** : Copier les requirements avant le code
3. **Utiliser `.dockerignore`** : Exclure les fichiers inutiles
4. **Minimiser les couches** : Combiner les commandes RUN liées
5. **Ne pas exécuter en root** : Créer un utilisateur non-root quand possible
6. **Utiliser les constructions multi-étapes** : Pour des images de production plus légères

Exemple `.dockerignore` :

```
__pycache__
*.pyc
*.pyo
.git
.gitignore
*.md
.env
venv/
.pytest_cache/
```

### Questions - Exercice 2

**Q2.1** Créez un Dockerfile pour une application PySpark qui :
- Utilise `bitnami/spark` comme image de base
- Installe des paquets Python supplémentaires (pandas, matplotlib)
- Copie un script Spark qui traite des données CSV
- Exécute le script au démarrage du conteneur

**Q2.2** Créez un Dockerfile qui :
- Utilise un utilisateur non-root pour la sécurité
- Implémente des vérifications de santé (health checks)
- Utilise des variables d'environnement pour la configuration
- Inclut un étiquetage correct (maintainer, version, description)

**Q2.3** Comparez les tailles d'images de :
- Un Dockerfile simple utilisant `python:3.10`
- La même application utilisant `python:3.10-slim`
- Une version avec construction multi-étapes

Documentez les différences de taille et expliquez quand chaque approche est appropriée.

---

## Exercice 3 : Docker Compose pour applications multi-conteneurs [★★]

### Aperçu de Docker Compose

Docker Compose vous permet de définir et d'exécuter des applications multi-conteneurs en utilisant un fichier YAML.

### Structure de base de docker-compose.yml

```yaml
version: "3.8"

services:
  nom_service:
    image: nom_image:tag
    # OU construire depuis un Dockerfile
    build: ./chemin/vers/dockerfile
    ports:
      - "port_hote:port_conteneur"
    volumes:
      - ./chemin/local:/chemin/conteneur
    environment:
      - NOM_VAR=valeur
    depends_on:
      - autre_service

volumes:
  volume_nomme:

networks:
  reseau_personnalise:
```

### Commandes Docker Compose

```bash
# Démarrer tous les services
docker-compose up

# Démarrer en mode détaché
docker-compose up -d

# Construire les images avant de démarrer
docker-compose up --build

# Arrêter tous les services
docker-compose down

# Arrêter et supprimer les volumes
docker-compose down -v

# Afficher les logs
docker-compose logs
docker-compose logs -f nom_service

# Mettre à l'échelle un service
docker-compose up --scale nom_service=3

# Exécuter une commande dans un service
docker-compose exec nom_service commande
```

### Exemple : Application Web avec Redis

Créez `app.py` :

```python
from flask import Flask
import redis

app = Flask(__name__)
cache = redis.Redis(host='redis', port=6379)

@app.route('/')
def hello():
    count = cache.incr('hits')
    return f'Bonjour ! Cette page a été vue {count} fois.'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

Créez `requirements.txt` :

```
flask
redis
```

Créez `Dockerfile` :

```dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app.py .

EXPOSE 5000

CMD ["python", "app.py"]
```

Créez `docker-compose.yml` :

```yaml
version: "3.8"

services:
  web:
    build: .
    ports:
      - "5000:5000"
    depends_on:
      - redis
    environment:
      - FLASK_ENV=development

  redis:
    image: redis:alpine
    volumes:
      - redis_data:/data

volumes:
  redis_data:
```

Exécutez avec :

```bash
docker-compose up --build
```

### Dépendances de services et vérifications de santé

```yaml
version: "3.8"

services:
  web:
    build: .
    depends_on:
      db:
        condition: service_healthy
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:5000/health"]
      interval: 30s
      timeout: 10s
      retries: 3

  db:
    image: postgres:15
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 5s
      timeout: 5s
      retries: 5
```

### Questions - Exercice 3

**Q3.1** Créez une configuration Docker Compose pour un pipeline de traitement de données avec :
- Un service générateur de données Python
- Un service Redis pour la mise en cache
- Un service processeur de données qui lit depuis Redis
- Des dépendances de services correctes

**Q3.2** Modifiez l'exemple précédent pour utiliser :
- Des réseaux personnalisés pour l'isolation des services
- Des fichiers d'environnement (`.env`)
- Des montages de volumes pour la persistance des données

**Q3.3** Créez un fichier Docker Compose qui démarre un serveur Jupyter Notebook avec :
- Des bibliothèques de science des données préinstallées (pandas, numpy, matplotlib, sklearn)
- Un stockage persistant des notebooks
- Accès à un volume de données partagé

---

## Exercice 4 : Pipelines de données avec volumes partagés [★★]

### Volumes partagés pour la communication entre conteneurs

Les volumes partagés permettent aux conteneurs d'échanger des données via le système de fichiers.

```
┌─────────────────┐       ┌─────────────────┐
│    Uploader     │       │    Processeur   │
│    Conteneur    │       │    Conteneur    │
│                 │       │                 │
│   écrit dans    │       │   lit depuis    │
│   /shared       │       │   /shared       │
└────────┬────────┘       └────────┬────────┘
         │                         │
         └─────────┬───────────────┘
                   │
            ┌──────┴──────┐
            │   Volume    │
            │   Partagé   │
            └─────────────┘
```

### Exemple : Pipeline de traitement de fichiers

Naviguez vers le dossier `SharedVolume` de ce TP :

```bash
cd SharedVolume
```

Examinez la structure existante :

**Service Uploader** (`Uploader/upload.py`) :
```python
import time
from shutil import copyfile

def upload_file():
    while True:
        # Simule l'upload d'un nouveau fichier toutes les 5 secondes
        print("Upload d'un nouveau fichier...")
        copyfile("sample.txt", "/shared/sample_uploaded.txt")
        time.sleep(5)

if __name__ == "__main__":
    upload_file()
```

**Service Processeur** (`Processor/process.py`) :
```python
import time
import os

def process_files():
    while True:
        if os.path.exists("/shared/sample_uploaded.txt"):
            with open("/shared/sample_uploaded.txt", "r") as f:
                content = f.read()
            print(f"Traitement : {content}")
            # Traiter le fichier...
            os.remove("/shared/sample_uploaded.txt")
        else:
            print("En attente de fichiers...")
        time.sleep(2)

if __name__ == "__main__":
    process_files()
```

**docker-compose.yml** :
```yaml
version: "3.8"

services:
  uploader:
    build:
      context: ./uploader
    volumes:
      - ./shared:/shared
    depends_on:
      - processor

  processor:
    build:
      context: ./processor
    volumes:
      - ./shared:/shared
```

Exécutez avec :
```bash
docker-compose up --build
```

### Exemple de pipeline de données amélioré

Créez un pipeline de données plus sophistiqué :

**data_generator.py** :
```python
import json
import time
import random
from datetime import datetime

def generate_data():
    counter = 0
    while True:
        data = {
            "id": counter,
            "timestamp": datetime.now().isoformat(),
            "sensor_id": f"capteur_{random.randint(1, 10)}",
            "temperature": round(random.uniform(20, 35), 2),
            "humidite": round(random.uniform(30, 80), 2)
        }
        
        filename = f"/shared/input/data_{counter}.json"
        with open(filename, 'w') as f:
            json.dump(data, f)
        
        print(f"Généré : {filename}")
        counter += 1
        time.sleep(2)

if __name__ == "__main__":
    import os
    os.makedirs("/shared/input", exist_ok=True)
    generate_data()
```

**data_processor.py** :
```python
import json
import os
import time

def process_files():
    os.makedirs("/shared/output", exist_ok=True)
    
    while True:
        input_dir = "/shared/input"
        if os.path.exists(input_dir):
            files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
            
            for filename in files:
                filepath = os.path.join(input_dir, filename)
                
                with open(filepath, 'r') as f:
                    data = json.load(f)
                
                # Traiter les données
                data['traite'] = True
                data['temp_fahrenheit'] = round(data['temperature'] * 9/5 + 32, 2)
                
                # Écrire dans le répertoire de sortie
                output_path = f"/shared/output/traite_{filename}"
                with open(output_path, 'w') as f:
                    json.dump(data, f, indent=2)
                
                # Supprimer le fichier d'entrée
                os.remove(filepath)
                print(f"Traité : {filename}")
        
        time.sleep(1)

if __name__ == "__main__":
    process_files()
```

### Questions - Exercice 4

**Q4.1** Étendez l'exemple SharedVolume pour :
- Ajouter un troisième service qui agrège les fichiers traités
- Générer des statistiques (température moyenne, humidité par capteur)
- Produire un rapport de synthèse chaque minute

**Q4.2** Implémentez la gestion des erreurs dans le pipeline :
- Déplacer les fichiers en échec vers un répertoire "erreur"
- Journaliser les erreurs avec des horodatages
- Ajouter un service de surveillance qui rapporte l'état du pipeline

**Q4.3** Créez un pipeline de traitement parallèle :
- Plusieurs conteneurs processeurs (utilisez `--scale`)
- Implémentez le verrouillage de fichiers pour éviter le traitement en double
- Mesurez le débit avec différents nombres de processeurs

---

## Exercice 5 : Producteur-Consommateur avec files de messages [★★]

### Modèle de file de messages

Les files de messages découplent les producteurs et consommateurs, permettant :
- Le traitement asynchrone
- L'équilibrage de charge
- La tolérance aux pannes

```
┌──────────┐     ┌─────────────┐     ┌──────────┐
│Producteur│────►│   File de   │────►│Consommat.│
│    1     │     │   Messages  │     │    1     │
└──────────┘     │  (RabbitMQ)│     └──────────┘
┌──────────┐     │             │     ┌──────────┐
│Producteur│────►│             │────►│Consommat.│
│    2     │     └─────────────┘     │    2     │
└──────────┘                         └──────────┘
```

### Exemple RabbitMQ

Naviguez vers le dossier `ProducerConsumerRabbitMQ` :

```bash
cd ProducerConsumerRabbitMQ
```

**producer/producer.py** :
```python
import pika
import time

def connect():
    for i in range(5):
        try:
            return pika.BlockingConnection(pika.ConnectionParameters('rabbitmq'))
        except:
            print("Nouvelle tentative de connexion à RabbitMQ...")
            time.sleep(2)
    raise Exception("Impossible de se connecter à RabbitMQ")

connection = connect()
channel = connection.channel()
channel.queue_declare(queue='task_queue', durable=True)

for i in range(100):
    msg = f"Tâche #{i}"
    channel.basic_publish(
        exchange='',
        routing_key='task_queue',
        body=msg,
        properties=pika.BasicProperties(delivery_mode=2)  # Rendre le message persistant
    )
    print(f"Envoyé : {msg}")
    time.sleep(1)

connection.close()
```

**consumer/consumer.py** :
```python
import pika
import time

def connect():
    for i in range(5):
        try:
            return pika.BlockingConnection(pika.ConnectionParameters('rabbitmq'))
        except:
            print("Nouvelle tentative de connexion à RabbitMQ...")
            time.sleep(2)
    raise Exception("Impossible de se connecter à RabbitMQ")

def callback(ch, method, properties, body):
    print(f"Reçu : {body.decode()}")
    time.sleep(0.5)  # Simuler le traitement
    print(f"Traité : {body.decode()}")
    ch.basic_ack(delivery_tag=method.delivery_tag)

connection = connect()
channel = connection.channel()
channel.queue_declare(queue='task_queue', durable=True)
channel.basic_qos(prefetch_count=1)  # Distribution équitable
channel.basic_consume(queue='task_queue', on_message_callback=callback)

print('En attente de messages...')
channel.start_consuming()
```

**docker-compose.yml** :
```yaml
services:
  rabbitmq:
    image: rabbitmq:3-management
    ports:
      - "5672:5672"   # Protocole AMQP
      - "15672:15672" # Interface de gestion
    environment:
      RABBITMQ_DEFAULT_USER: guest
      RABBITMQ_DEFAULT_PASS: guest

  producer:
    build: ./producer
    depends_on:
      - rabbitmq

  consumer:
    build: ./consumer
    depends_on:
      - rabbitmq
```

Exécutez avec :
```bash
docker-compose up --build

# Mettre à l'échelle les consommateurs
docker-compose up --scale consumer=3
```

Accédez à l'interface de gestion RabbitMQ : http://localhost:15672 (guest/guest)

### Traitement de données avec files de messages

Producteur amélioré pour le traitement de données :

```python
# data_producer.py
import pika
import json
import random
import time
from datetime import datetime

def connect():
    for i in range(5):
        try:
            return pika.BlockingConnection(pika.ConnectionParameters('rabbitmq'))
        except:
            time.sleep(2)
    raise Exception("Impossible de se connecter")

connection = connect()
channel = connection.channel()
channel.queue_declare(queue='data_queue', durable=True)

sensors = ['temperature', 'humidite', 'pression']

while True:
    data = {
        'type_capteur': random.choice(sensors),
        'valeur': round(random.uniform(0, 100), 2),
        'timestamp': datetime.now().isoformat()
    }
    
    channel.basic_publish(
        exchange='',
        routing_key='data_queue',
        body=json.dumps(data),
        properties=pika.BasicProperties(delivery_mode=2)
    )
    
    print(f"Envoyé : {data}")
    time.sleep(0.5)
```

### Questions - Exercice 5

**Q5.1** Étendez l'exemple RabbitMQ pour :
- Utiliser le routage par sujet (différentes files pour différents types de données)
- Implémenter plusieurs types de consommateurs (un pour chaque type de capteur)
- Stocker les données traitées dans un volume partagé

**Q5.2** Implémentez la gestion des lettres mortes :
- Configurez une file de lettres mortes pour les messages échoués
- Ajoutez un mécanisme de nouvelle tentative (maximum 3 tentatives)
- Créez un consommateur de surveillance qui alerte sur les messages DLQ

**Q5.3** Comparez RabbitMQ avec Redis Pub/Sub :
- Implémentez le même modèle producteur-consommateur avec Redis
- Mesurez le débit des messages
- Documentez les compromis entre les deux approches

---

## Exercice 6 : Intégration Application-Base de données [★★]

### Connexion d'applications aux bases de données

Naviguez vers le dossier `AppDB` :

```bash
cd AppDB
```

Cet exemple démontre une application Flask connectée à PostgreSQL.

**app/app.py** :
```python
from flask import Flask
import psycopg2

app = Flask(__name__)

@app.route("/")
def index():
    conn = psycopg2.connect(
        host="bd",  # Nom du service dans Docker
        database="livres",
        user="postgres",
        password="postgres"
    )
    cur = conn.cursor()
    cur.execute("SELECT titre FROM livres")
    livres = cur.fetchall()
    cur.close()
    conn.close()
    return "<br>".join(title for (title,) in livres)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
```

**init_bd/init.sql** :
```sql
CREATE TABLE IF NOT EXISTS livres (
    id SERIAL PRIMARY KEY,
    titre VARCHAR(255) NOT NULL,
    auteur VARCHAR(255),
    annee INTEGER
);

INSERT INTO livres (titre, auteur, annee) VALUES
    ('Les Misérables', 'Victor Hugo', 1862),
    ('Le Petit Prince', 'Antoine de Saint-Exupéry', 1943),
    ('L''Étranger', 'Albert Camus', 1942);
```

**docker-compose.yml** :
```yaml
services:
  app:
    build: ./app
    ports:
      - "5000:5000"
    depends_on:
      - bd

  bd:
    image: postgres:15
    environment:
      POSTGRES_DB: livres
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: postgres
    volumes:
      - ./init_bd:/docker-entrypoint-initdb.d
      - postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

volumes:
  postgres_data:
```

Exécutez avec :
```bash
docker-compose up --build
```

Accédez à : http://localhost:5000

### Exemple amélioré avec SQLAlchemy

```python
# app_enhanced.py
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)

# Configuration de la base de données depuis l'environnement
db_host = os.environ.get('DB_HOST', 'bd')
db_name = os.environ.get('DB_NAME', 'livres')
db_user = os.environ.get('DB_USER', 'postgres')
db_pass = os.environ.get('DB_PASS', 'postgres')

app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{db_user}:{db_pass}@{db_host}/{db_name}'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Book(db.Model):
    __tablename__ = 'livres'
    id = db.Column(db.Integer, primary_key=True)
    titre = db.Column(db.String(255), nullable=False)
    auteur = db.Column(db.String(255))
    annee = db.Column(db.Integer)

@app.route('/books')
def get_books():
    books = Book.query.all()
    return jsonify([{
        'id': b.id,
        'titre': b.titre,
        'auteur': b.auteur,
        'annee': b.annee
    } for b in books])

@app.route('/books', methods=['POST'])
def add_book():
    data = request.json
    book = Book(titre=data['titre'], auteur=data['auteur'], annee=data['annee'])
    db.session.add(book)
    db.session.commit()
    return jsonify({'id': book.id}), 201

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

### Questions - Exercice 6

**Q6.1** Étendez l'exemple AppDB pour inclure :
- Opérations CRUD (Créer, Lire, Mettre à jour, Supprimer)
- Validation des entrées
- Gestion des erreurs avec codes de statut HTTP appropriés

**Q6.2** Ajoutez des capacités d'analyse de données :
- Endpoint pour obtenir les livres par plage d'années
- Endpoint de statistiques (nombre par auteur, livres par décennie)
- Capacité de recherche en texte intégral

**Q6.3** Implémentez un service d'importation de données :
- Créez un conteneur séparé qui importe des données CSV dans la base de données
- Surveillez un volume partagé pour les nouveaux fichiers CSV
- Journalisez les résultats et erreurs d'importation

---

## Exercice 7 : Architectures Frontend-Backend [★★★]

### Architecture Microservices

Naviguez vers le dossier `WebAppFrontBack` :

```bash
cd WebAppFrontBack
```

Cet exemple démontre un frontend React avec un backend Flask.

```
┌─────────────────┐      ┌─────────────────┐
│    Frontend     │      │    Backend      │
│    (React)      │─────►│    (Flask)      │
│   Port: 3000   │      │   Port: 5000    │
└─────────────────┘      └─────────────────┘
```

### API Backend (Flask)

**backend/app.py** :
```python
from flask import Flask, jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Activer les requêtes Cross-Origin

# Stockage de données en mémoire
tasks = [
    {"id": 1, "title": "Apprendre Docker", "completed": True},
    {"id": 2, "title": "Construire un pipeline", "completed": False}
]

@app.route('/api/tasks', methods=['GET'])
def get_tasks():
    return jsonify(tasks)

@app.route('/api/tasks', methods=['POST'])
def add_task():
    data = request.json
    new_task = {
        "id": len(tasks) + 1,
        "title": data['title'],
        "completed": False
    }
    tasks.append(new_task)
    return jsonify(new_task), 201

@app.route('/api/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = next((t for t in tasks if t['id'] == task_id), None)
    if task:
        data = request.json
        task['completed'] = data.get('completed', task['completed'])
        return jsonify(task)
    return jsonify({"error": "Tâche non trouvée"}), 404

@app.route('/api/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    global tasks
    tasks = [t for t in tasks if t['id'] != task_id]
    return '', 204

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

### Docker Compose pour Full Stack

**docker-compose.yml** :
```yaml
version: "3.8"

services:
  frontend:
    build:
      context: ./frontend
    ports:
      - "3000:3000"
    depends_on:
      - backend
    environment:
      - REACT_APP_API_URL=http://localhost:5000

  backend:
    build:
      context: ./backend
    ports:
      - "5000:5000"
    volumes:
      - ./backend:/app
    environment:
      - FLASK_ENV=development
```

### Ajout de Nginx comme Reverse Proxy

Pour les déploiements en production, utilisez Nginx comme reverse proxy :

**nginx.conf** :
```nginx
upstream frontend {
    server frontend:3000;
}

upstream backend {
    server backend:5000;
}

server {
    listen 80;

    location / {
        proxy_pass http://frontend;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection "upgrade";
    }

    location /api {
        proxy_pass http://backend;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

**docker-compose.prod.yml** :
```yaml
version: "3.8"

services:
  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/conf.d/default.conf
    depends_on:
      - frontend
      - backend

  frontend:
    build:
      context: ./frontend
      dockerfile: Dockerfile.prod

  backend:
    build:
      context: ./backend
```

### Questions - Exercice 7

**Q7.1** Étendez l'exemple frontend-backend pour inclure :
- Authentification utilisateur (connexion/déconnexion)
- Routes protégées
- Gestion des tokens JWT

**Q7.2** Ajoutez une base de données à la pile :
- Remplacez le stockage en mémoire par PostgreSQL
- Ajoutez des migrations de base de données
- Implémentez la persistance des données entre les redémarrages

**Q7.3** Créez un tableau de bord de visualisation de données :
- API backend qui sert des données d'analyse
- Frontend avec des graphiques (utilisant Chart.js ou similaire)
- Mises à jour en temps réel utilisant WebSockets

---

## Exercice 8 : Mise à l'échelle et surveillance des conteneurs [★★★]

### Mise à l'échelle des conteneurs

```bash
# Mettre à l'échelle un service spécifique
docker-compose up --scale worker=5

# Afficher les conteneurs en cours d'exécution
docker-compose ps

# Afficher l'utilisation des ressources
docker stats
```

### Équilibrage de charge avec Nginx

**docker-compose.yml** :
```yaml
version: "3.8"

services:
  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - api

  api:
    build: .
    # Pas de ports exposés - accès via nginx
    deploy:
      replicas: 3
```

**nginx.conf** pour l'équilibrage de charge :
```nginx
events {
    worker_connections 1024;
}

http {
    upstream api_servers {
        least_conn;  # Méthode d'équilibrage de charge
        server api:5000;
    }

    server {
        listen 80;

        location / {
            proxy_pass http://api_servers;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
        }
    }
}
```

### Surveillance avec Prometheus et Grafana

**docker-compose.monitoring.yml** :
```yaml
version: "3.8"

services:
  prometheus:
    image: prom/prometheus
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
      - prometheus_data:/prometheus

  grafana:
    image: grafana/grafana
    ports:
      - "3000:3000"
    volumes:
      - grafana_data:/var/lib/grafana
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin

  cadvisor:
    image: gcr.io/cadvisor/cadvisor
    ports:
      - "8080:8080"
    volumes:
      - /:/rootfs:ro
      - /var/run:/var/run:ro
      - /sys:/sys:ro
      - /var/lib/docker/:/var/lib/docker:ro

volumes:
  prometheus_data:
  grafana_data:
```

**prometheus.yml** :
```yaml
global:
  scrape_interval: 15s

scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']

  - job_name: 'cadvisor'
    static_configs:
      - targets: ['cadvisor:8080']
```

### Limites de ressources

```yaml
version: "3.8"

services:
  api:
    build: .
    deploy:
      resources:
        limits:
          cpus: '0.50'
          memory: 512M
        reservations:
          cpus: '0.25'
          memory: 256M
```

### Questions - Exercice 8

**Q8.1** Créez un pipeline de traitement de données évolutif :
- Service producteur générant des données
- Services workers pouvant être mis à l'échelle (1-10 instances)
- Équilibreur de charge distribuant le travail
- Mesurez le débit avec différents nombres de workers

**Q8.2** Configurez la surveillance pour votre application :
- Configurez Prometheus pour collecter les métriques
- Créez des tableaux de bord Grafana pour :
  - Utilisation CPU et mémoire
  - Taux de requêtes et latences
  - Taux d'erreurs

**Q8.3** Implémentez une simulation d'auto-scaling :
- Surveillez l'utilisation CPU des conteneurs workers
- Créez un script qui met à l'échelle les workers en fonction de la charge
- Testez avec différents modèles de charge

---

## Résumé

Dans ce TP, vous avez appris :

1. **Fondamentaux Docker** : Images, conteneurs et commandes de base
2. **Dockerfiles** : Écriture de Dockerfiles efficaces pour applications Python
3. **Docker Compose** : Orchestration d'applications multi-conteneurs
4. **Volumes partagés** : Construction de pipelines de données avec communication par fichiers
5. **Files de messages** : Modèles producteur-consommateur avec RabbitMQ
6. **Intégration de base de données** : Connexion d'applications à PostgreSQL
7. **Frontend-Backend** : Construction d'applications full-stack
8. **Mise à l'échelle et surveillance** : Équilibrage de charge et observabilité

### Points clés à retenir

- Utilisez Docker Compose pour le développement et les tests
- Implémentez des vérifications de santé correctes pour les dépendances de services
- Utilisez des volumes pour la persistance des données
- Choisissez le bon modèle de communication (fichiers, messages, API)
- Surveillez et mettez à l'échelle en fonction des métriques

### Prochaines étapes

Dans le TP 7, vous apprendrez Kubernetes pour :
- L'orchestration de conteneurs de niveau production
- La gestion de configuration déclarative
- La mise à l'échelle automatique et l'auto-réparation
- La découverte de services et l'équilibrage de charge

### Lectures complémentaires

- [Documentation Docker](https://docs.docker.com/)
- [Documentation Docker Compose](https://docs.docker.com/compose/)
- [Bonnes pratiques pour l'écriture de Dockerfiles](https://docs.docker.com/develop/develop-images/dockerfile_best-practices/)
- [Bonnes pratiques de sécurité Docker](https://docs.docker.com/develop/security-best-practices/)